<a href="https://colab.research.google.com/github/yuthika21z/Deep-learning/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn.utils.rnn import pad_sequence # Add padding to sequences
# Sample data: Tokenized sentences and corresponding part-of-speech tags
# Replace with your own dataset
sentences = [
["The", "quick", "brown", "fox", "jumps", "over", "the", "lazy", "dog"],
["She", "sells", "seashells", "by", "the", "seashore"],
]
tags = [
["DT", "JJ", "JJ", "NN", "VBZ", "IN", "DT", "JJ", "NN"],

  ["PRP", "VBZ", "NNS", "IN", "DT", "NN"],
]
# Create a vocabulary and mapping for words and tags
word_to_idx = {word: idx for idx, word in enumerate(set(word for sentence in sentences for word in sentence))}
tag_to_idx = {tag: idx for idx, tag in enumerate(set(tag for tag_seq in tags for tag in tag_seq))}
# Convert words and tags to numerical indices
X = [[word_to_idx[word] for word in sentence] for sentence in sentences]
y = [[tag_to_idx[tag] for tag in tag_seq] for tag_seq in tags]
# Get the maximum sequence length
max_seq_length = max(len(seq) for seq in X)
# Pad sequences to the maximum length
X_padded = [torch.tensor(seq + [0] * (max_seq_length - len(seq))) for seq in X]
y_padded = [torch.tensor(seq + [0] * (max_seq_length - len(seq))) for seq in y]
# Convert the padded sequences to tensors
X_tensor = torch.stack(X_padded)
y_tensor = torch.stack(y_padded)
# Hyperparameters
embedding_dim = 50
hidden_dim = 64
vocab_size = len(word_to_idx)
tagset_size = len(tag_to_idx)
num_epochs = 100
learning_rate = 0.01
# Create DataLoader for batching
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [3]:
# Define the RNN model (same as before)
class RNNSequenceLabeling(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, tagset_size):
        super(RNNSequenceLabeling, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, tagset_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = self.fc(out)
        return out
# Create DataLoader for batching (same as before)
# Initialize and train the RNN model (same as before)
model = RNNSequenceLabeling(vocab_size, embedding_dim, hidden_dim, tagset_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(num_epochs):
    for batch_X, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs.view(-1, tagset_size), batch_y.view(-1))
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
# Test the trained model on a sample sentence (same as before)
test_sentence = ["She", "jumps", "over", "the", "lazy", "dog"]
test_input = torch.tensor([[word_to_idx[word] for word in test_sentence]])

with torch.no_grad():
   predicted_tags = model(test_input)
   predicted_tags = torch.argmax(predicted_tags, dim=2).squeeze().tolist()
# Map numerical tags back to labels
idx_to_tag = {idx: tag for tag, idx in tag_to_idx.items()}
predicted_tags = [idx_to_tag[idx] for idx in predicted_tags]
# Print the result
print("Test Sentence:", test_sentence)
print("Predicted Tags:", predicted_tags)



Epoch [10/100], Loss: 1.7656
Epoch [20/100], Loss: 1.6172
Epoch [30/100], Loss: 1.4812
Epoch [40/100], Loss: 1.3552
Epoch [50/100], Loss: 1.2379
Epoch [60/100], Loss: 1.1286
Epoch [70/100], Loss: 1.0269
Epoch [80/100], Loss: 0.9328
Epoch [90/100], Loss: 0.8461
Epoch [100/100], Loss: 0.7669
Test Sentence: ['She', 'jumps', 'over', 'the', 'lazy', 'dog']
Predicted Tags: ['PRP', 'VBZ', 'IN', 'DT', 'JJ', 'NN']
